# 1.Cargamos todos los datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import pandas as pd
from scipy import stats
import re
from datetime import datetime
import sklearn
from sklearn import set_config
from sklearn import model_selection
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer


In [ ]:
#datapath = "/content/drive/MyDrive/TFM/Dades/" # path Medir Vilà
datapath = "/content/drive/MyDrive/MASTER DATA SCIENCE 2023/TFM MASTER/EASY MONEY/datasets_TFM + diccionario" # path jaume

In [ ]:
customer_comercial_activity = pd.read_csv(datapath + 'customer_commercial_activity.csv')
customer_products = pd.read_csv(datapath + 'customer_products.csv')
customer_sociodemographics = pd.read_csv(datapath + 'customer_sociodemographics.csv')
product_description = pd.read_csv(datapath + 'product_description.csv')
sales = pd.read_csv(datapath + 'sales.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MASTER DATA SCIENCE 2023/TFM MASTER/EASY MONEY/datasets_TFM + diccionariocustomer_commercial_activity.csv'

In [ ]:

# Eliminar columnas innecesarias como 'Unnamed: 0'
for df in [customer_sociodemographics, customer_products, customer_comercial_activity, sales, product_description]:
    df.drop(columns=[col for col in df.columns if "Unnamed" in col], inplace=True)

# Renombrar columnas en 'sales' para consistencia
sales.rename(columns={'cid': 'pk_cid', 'product_ID': 'pk_product_ID'}, inplace=True)

# Comprobar si hay columnas duplicadas en 'sales' o 'product_description'
print("Columnas en Sales después de ajustar:", sales.columns)
print("Columnas en Product Description después de ajustar:", product_description.columns)

# Merge de datos de clientes (sociodemográficos, productos y actividad comercial)
customer_data = pd.merge(customer_sociodemographics, customer_products, on=['pk_cid', 'pk_partition'], how='left')
customer_data = pd.merge(customer_data, customer_comercial_activity, on=['pk_cid', 'pk_partition'], how='left')

# Corregir cualquier duplicidad en 'pk_product_ID' antes de hacer el merge
if sales.columns.duplicated().any():
    sales = sales.loc[:, ~sales.columns.duplicated()]

# Merge de descripciones de productos con ventas
sales = pd.merge(sales, product_description, on='pk_product_ID', how='left')

# Merge final incorporando los datos de ventas, usando un left join para mantener todos los clientes
final_data = pd.merge(customer_data, sales, on='pk_cid', how='left')

# Eliminar duplicados basándonos en las claves principales si es necesario
final_data.drop_duplicates(subset=['pk_cid', 'pk_sale'], inplace=True)

# Visualización de los primeros registros para verificar la integridad del merge
print(final_data.head())


NameError: name 'customer_sociodemographics' is not defined

In [ ]:
final_data

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,short_term_deposit,loans,...,entry_date,entry_channel,active_customer,segment,pk_sale,month_sale,pk_product_ID,net_margin,product_desc,family_product
0,1375586,2018-01,ES,29.0,H,35,N,87218.10,0,0,...,2018-01,KHL,1.0,02 - PARTICULARES,111967.0,2018-05-01,3819.0,56.9,debit_card,payment_card
1,1050611,2018-01,ES,13.0,V,23,N,35548.74,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaN,NaN,NaN,NaN,NaN
2,1050612,2018-01,ES,13.0,V,23,N,122179.11,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaN,NaN,NaN,NaN,NaN
3,1050613,2018-01,ES,50.0,H,22,N,119775.54,1,0,...,2015-08,KHD,0.0,03 - UNIVERSITARIO,NaN,NaN,NaN,NaN,NaN,NaN
4,1050614,2018-01,ES,50.0,V,23,N,NaN,0,0,...,2015-08,KHE,1.0,03 - UNIVERSITARIO,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7031189,1185013,2019-05,ES,23.0,V,53,N,NaN,0,0,...,2019-05,NaN,1.0,NaN,135643.0,2019-05-01,4657.0,60.7,em_acount,account
7040744,1168909,2019-05,ES,36.0,V,43,N,NaN,0,0,...,2016-08,NaN,1.0,NaN,135263.0,2019-05-01,4657.0,76.0,em_acount,account
7042025,1173729,2019-05,ES,14.0,H,33,N,NaN,0,0,...,2016-09,NaN,1.0,NaN,135375.0,2019-05-01,4657.0,59.7,em_acount,account
7044808,1164094,2019-05,ES,30.0,H,54,N,NaN,0,0,...,2019-05,KFC,1.0,NaN,135165.0,2019-05-01,4657.0,86.8,em_acount,account


In [ ]:
# Suma de net margin
nombre_columna = 'net_margin'
suma_columna = final_data[nombre_columna].sum()
print("La suma de la columna", nombre_columna, "es:", suma_columna)

La suma de la columna net_margin es: 146119876.29999998


In [ ]:

# Rellenar nulos en columnas numéricas con la mediana
numeric_columns = ['salary']
for column in numeric_columns:
    final_data[column].fillna(final_data[column].median(), inplace=True)

# Rellenar nulos en columnas categóricas con la moda (el valor más frecuente)
categorical_columns = ['segment']
for column in categorical_columns:
    mode_value = final_data[column].mode()[0]
    final_data[column].fillna(mode_value, inplace=True)

# Eliminar filas donde aún haya nulos si son críticos (opcional)
final_data.dropna(subset=['pk_cid', 'pk_partition'], inplace=True)

# Convertir fechas a formato datetime
final_data['pk_partition'] = pd.to_datetime(final_data['pk_partition'])
final_data['month_sale'] = pd.to_datetime(final_data['month_sale'])

# Revisar el DataFrame limpio
print(final_data.info())
print(final_data.head())


<class 'pandas.core.frame.DataFrame'>
Index: 544392 entries, 0 to 7047285
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pk_cid              544392 non-null  int64         
 1   pk_partition        544392 non-null  datetime64[ns]
 2   country_id          544392 non-null  object        
 3   region_code         544207 non-null  float64       
 4   gender              544389 non-null  object        
 5   age                 544392 non-null  int64         
 6   deceased            544392 non-null  object        
 7   salary              544392 non-null  float64       
 8   short_term_deposit  544392 non-null  int64         
 9   loans               544392 non-null  int64         
 10  mortgage            544392 non-null  int64         
 11  funds               544392 non-null  int64         
 12  securities          544392 non-null  int64         
 13  long_term_deposit   544392 non-nu

In [ ]:
final_data.head(100)

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,short_term_deposit,loans,...,entry_date,entry_channel,active_customer,segment,pk_sale,month_sale,pk_product_ID,net_margin,product_desc,family_product
0,1375586,2018-01-01,ES,29.0,H,35,N,87218.100,0,0,...,2018-01,KHL,1.0,02 - PARTICULARES,111967.0,2018-05-01,3819.0,56.9,debit_card,payment_card
1,1050611,2018-01-01,ES,13.0,V,23,N,35548.740,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
2,1050612,2018-01-01,ES,13.0,V,23,N,122179.110,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
3,1050613,2018-01-01,ES,50.0,H,22,N,119775.540,1,0,...,2015-08,KHD,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
4,1050614,2018-01-01,ES,50.0,V,23,N,89158.785,0,0,...,2015-08,KHE,1.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1050523,2018-01-01,ES,46.0,H,24,N,77383.980,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
96,1050524,2018-01-01,ES,27.0,V,25,N,56347.680,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
97,1050505,2018-01-01,ES,45.0,V,25,N,112330.350,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN
98,1050525,2018-01-01,ES,28.0,H,26,N,95392.710,0,0,...,2015-08,KHE,0.0,03 - UNIVERSITARIO,NaN,NaT,NaN,NaN,NaN,NaN


In [ ]:
#iteramos sobre los nulos que tenemos :

### Iteramos los nulos

In [ ]:
columns_to_replace = ['entry_channel']
for column in columns_to_replace:
    final_data[column].fillna("no data", inplace=True)

In [ ]:
columns_to_replace = ['pk_sale', 'pk_product_ID', 'net_margin',  ]
for column in columns_to_replace:
    final_data[column].fillna(0, inplace=True)

In [ ]:
columns_to_replace = ['month_sale']
for column in columns_to_replace:
    final_data[column].fillna('2021-01-01', inplace=True)

In [ ]:
columns_to_replace = ['product_desc' , 'family_product']
for column in columns_to_replace:
    final_data[column].fillna('No Product', inplace=True)

In [ ]:
columns_to_replace = ['region_code']
for column in columns_to_replace:
    final_data[column].fillna(0, inplace=True)

In [ ]:
columns_to_replace = ['gender']
for column in columns_to_replace:
    final_data[column].fillna("H", inplace=True)

In [ ]:
columns_to_replace = ['payroll', 'pension_plan']
for column in columns_to_replace:
    final_data[column].fillna(0, inplace=True)

In [ ]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 544392 entries, 0 to 7047285
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pk_cid              544392 non-null  int64         
 1   pk_partition        544392 non-null  datetime64[ns]
 2   country_id          544392 non-null  object        
 3   region_code         544392 non-null  float64       
 4   gender              544392 non-null  object        
 5   age                 544392 non-null  int64         
 6   deceased            544392 non-null  object        
 7   salary              544392 non-null  float64       
 8   short_term_deposit  544392 non-null  int64         
 9   loans               544392 non-null  int64         
 10  mortgage            544392 non-null  int64         
 11  funds               544392 non-null  int64         
 12  securities          544392 non-null  int64         
 13  long_term_deposit   544392 non-nu

In [ ]:
final_data.to_csv(datapath + 'final_data_con_ventas.csv', index=True)